將IDW計算出的台灣空汙擴散圖整合成鄉鎮週暴露量（含ID欄位）

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [2]:
# 測項清單
factors = ["NO", "NO2", "NOx", "O3", "PM10", "PM25", "SO2"]

# === 1️⃣ 載入鄉鎮邊界（名稱欄位） ===
gml_path = "TOWN_MOI_1131028.gml"
taiwan_map = gpd.read_file(gml_path)
taiwan_map = taiwan_map.set_crs("EPSG:3824").to_crs("EPSG:4326")
taiwan_map = taiwan_map.rename(columns={"名稱": "town"})

# === 2️⃣ 載入鄉鎮對應ID ===
id_map = pd.read_csv("ID_CNAME.csv", dtype=str)
id_map = id_map.rename(columns={"ID1_CITY": "ID", "C_NAME": "town"})

# === 3️⃣ 建立 120x120 格點 ===
iInterval = (122.7 - 119) / 120
jInterval = (25.5 - 21.8) / 120
grid_points = [Point(119 + i * iInterval, 25.5 - j * jInterval) for j in range(120) for i in range(120)]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")

# === 4️⃣ 儲存所有結果（主鍵為 town, year, week） ===
all_results = {}

In [ ]:
# === 5️⃣ 處理每個測項 ===
for factor in factors:
    input_folder = f"./5_grid_output/{factor}"
    if not os.path.exists(input_folder):
        print(f"❌ 資料夾不存在: {input_folder}")
        continue

    for year in range(2015, 2020):
        for week in range(1, 52):
            grid_path = os.path.join(input_folder, f"{factor}_{year}_week_{week}.csv")
            if not os.path.exists(grid_path):
                continue

            try:
                grid_values = pd.read_csv(grid_path, skiprows=1, header=None).values.flatten()
            except Exception as e:
                print(f"❌ 無法讀取 {grid_path}: {e}")
                continue

            if len(grid_values) != len(grid_gdf):
                print(f"⚠️ 長度不符：{grid_path} ({len(grid_values)} vs {len(grid_gdf)})")
                continue

            valid_mask = grid_values != -1
            if valid_mask.sum() == 0:
                print(f"⚠️ 全為 -1：{grid_path}")
                continue

            grid_subset = grid_gdf[valid_mask].copy()
            grid_subset["value"] = grid_values[valid_mask]

            # 空間連接：格點 → 鄉鎮
            joined = gpd.sjoin(grid_subset, taiwan_map[["geometry", "town"]], predicate='within', how='inner')

            # 計算每個鄉鎮覆蓋比例（格點數法）
            grouped = joined.groupby('town')

            for town, group in grouped:
                # 該鄉鎮的總格點數（所有格點落在該鄉鎮內）
                total_grid_in_town = len(grid_gdf[grid_gdf.within(taiwan_map.loc[taiwan_map["town"] == town, "geometry"].values[0])])

                # 有值的格點數
                valid_grid_in_town = len(group)

                coverage_ratio = valid_grid_in_town / total_grid_in_town

                # 若覆蓋不到一半 → 略過
                if coverage_ratio < 0.5:
                    print(f"⚠️{year} {week} {town} 覆蓋不足（{coverage_ratio:.2%}），略過。")
                    continue

                print(f"✅{year} {week}  {town} 覆蓋充足（{coverage_ratio:.2%}），計算。")

                # 計算週暴露量（7天平均）
                avg_val = round(group["value"].mean(), 2)
                key = (town, year, week)
                if key not in all_results:
                    all_results[key] = {}
                all_results[key][factor] = avg_val

            print(f"✅ 完成：{factor} {year} week {week}")


✅2015 1  南投縣中寮鄉 覆蓋充足（97.70%），計算。
⚠️2015 1 南投縣仁愛鄉 覆蓋不足（5.76%），略過。
⚠️2015 1 南投縣信義鄉 覆蓋不足（1.65%），略過。
✅2015 1  南投縣南投市 覆蓋充足（114.11%），計算。
✅2015 1  南投縣名間鄉 覆蓋充足（118.21%），計算。
✅2015 1  南投縣國姓鄉 覆蓋充足（68.26%），計算。
✅2015 1  南投縣埔里鎮 覆蓋充足（95.67%），計算。
✅2015 1  南投縣水里鄉 覆蓋充足（62.89%），計算。
✅2015 1  南投縣竹山鎮 覆蓋充足（63.42%），計算。
✅2015 1  南投縣草屯鎮 覆蓋充足（90.14%），計算。
✅2015 1  南投縣集集鎮 覆蓋充足（88.45%），計算。
✅2015 1  南投縣魚池鄉 覆蓋充足（65.93%），計算。
✅2015 1  南投縣鹿谷鄉 覆蓋充足（81.95%），計算。
✅2015 1  嘉義市東區 覆蓋充足（78.49%），計算。
✅2015 1  嘉義市西區 覆蓋充足（156.84%），計算。
✅2015 1  嘉義縣中埔鄉 覆蓋充足（90.00%），計算。
✅2015 1  嘉義縣六腳鄉 覆蓋充足（112.78%），計算。
✅2015 1  嘉義縣大埔鄉 覆蓋充足（100.81%），計算。
✅2015 1  嘉義縣大林鎮 覆蓋充足（108.78%），計算。
✅2015 1  嘉義縣太保市 覆蓋充足（123.39%），計算。
✅2015 1  嘉義縣布袋鎮 覆蓋充足（88.91%），計算。
✅2015 1  嘉義縣新港鄉 覆蓋充足（106.18%），計算。
✅2015 1  嘉義縣朴子市 覆蓋充足（93.78%），計算。
✅2015 1  嘉義縣東石鄉 覆蓋充足（100.02%），計算。
✅2015 1  嘉義縣梅山鄉 覆蓋充足（58.82%），計算。
✅2015 1  嘉義縣民雄鄉 覆蓋充足（96.59%），計算。
✅2015 1  嘉義縣水上鄉 覆蓋充足（83.60%），計算。
✅2015 1  嘉義縣溪口鄉 覆蓋充足（144.53%），計算。
✅2015 1  嘉義縣番路鄉 覆蓋充足（83.37%），計算。
✅2015 1  嘉義縣竹崎鄉 覆蓋充足（59.75%），計算。
✅201

KeyboardInterrupt: 

In [ ]:
# === 6️⃣ 組裝整合表 ===
records = []
for (town, year, week), values in all_results.items():
    record = {"town": town, "year": year, "week": week}
    for factor in factors:
        record[factor] = values.get(factor, None)
    records.append(record)

result_df = pd.DataFrame(records)

# === 7️⃣ 合併ID對照 ===
result_df = result_df.merge(id_map, on="town", how="left")

# 將 ID 欄位移到第一欄
cols = ["ID"] + [c for c in result_df.columns if c != "ID"]
result_df = result_df[cols]

# 警告：有未對應的鄉鎮
missing_ids = result_df[result_df["ID"].isna()]["town"].unique()
if len(missing_ids) > 0:
    print("⚠️ 下列鄉鎮無法在 ID_CNAME.csv 中找到對應：")
    for name in missing_ids:
        print("  -", name)

# === 8️⃣ 輸出結果 ===
output_folder = "./6_exposure_by_town"
os.makedirs(output_folder, exist_ok=True)

result_df = result_df.sort_values(by=["ID", "year", "week"])
result_df.to_csv(f"{output_folder}/factors_weekly_exposure_with_ID.csv", index=False, encoding="utf-8-sig")

print("✅ 最終輸出完成：6_exposure_by_town/factors_weekly_exposure_with_ID.csv")